<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/Sihyun/data_exploration_judges(fatique_April_20).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Change the current directory (dataset)
%cd /content/drive/MyDrive/School/MADS/SIADS 699 : Capstone/Team

/content/drive/MyDrive/School/MADS/SIADS 699 : Capstone/Team


Import libraries

In [3]:
import pandas as pd
import numpy as np
import altair as alt
import re
from datetime import datetime, timedelta
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns
from scipy import stats
from natsort import natsorted


In [4]:
# altair library. In case you may need it
!pip install --upgrade altair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2


In [5]:
#!ls

In [6]:
# Load all sheets into a dictionary of DataFrames
webscraping = pd.read_excel('Webscraping.xlsx', sheet_name=None)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
webscraping.keys()

dict_keys(['Events and8', 'Events', 'Events WDSF', 'Battles Trivium', 'Battles and8 Threefold', 'Battles and8 Single Slider', 'Battles and8 Round by Round', 'Battles and8 Traditional', 'Research Questions', 'Notes', 'Notes Columns'])

# Threefold


---
Between each round of a battle, each judge gives a score (1-100) representing how much they believe one side outperformed the other (or scores 0 to represent a tie) in each of 3 categories (physical, artistic, interpretive), the sum of those 3 scores determine which side the judge is voting for, and the majority of judge votes determines the winner of the round.


In [8]:
#Load battle winners
#threefold
threefold = webscraping['Battles and8 Threefold']
threefold.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category', 'breaker 1': 'breaker 1 (red)', 'breaker 2': 'breaker 2 (blue)'}, inplace=True)
threefold[['Event', 'Category']] = threefold[['Event', 'Category']].fillna(method='ffill')
#Trivium
trivium=webscraping['Battles Trivium']
trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
trivium[['Event', 'Category']]= trivium[['Event', 'Category']].fillna(method='ffill')
#traditional
traditional = webscraping['Battles and8 Traditional']
traditional.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category', 'breaker 1': 'breaker 1 (red)', 'breaker 2': 'breaker 2 (blue)'}, inplace=True)
traditional[['Event', 'Category']] = traditional[['Event', 'Category']].fillna(method='ffill')

# Traditional


---

At the end of a battle, each judge points to the left or right towards the side they individually believe to have won the battle (or makes an x with their arms to represent a tie), and the majority determines the winner. In the case of a tie, generally the competitors perform one more round each, and then the judges vote again based on the additional round. This is the most widespread judging system used from local events through international competitions because of its simplicity in implementation and no need for additional technology setup.

In [9]:
# Load traditional
tr=traditional[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()
th=threefold[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()
tv=trivium[['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner']].copy()

dataframes = [('traditional', tr), ('threefold', th), ('trivium', tv)]

# Create an empty list to store modified dataframes
appended_dfs = []

# Iterate to create a combined dataframe
for name, dframe in dataframes:
    dframe['Original_DF'] = name
    dframe['Battle_ID'] = dframe.index
    appended_dfs.append(dframe)
appendedBattles = pd.concat(appended_dfs, ignore_index=True)

In [10]:
# Load And8
eventsAnd8=webscraping['Events and8'][1:].copy()
new_header= eventsAnd8.iloc[0]
eventsAnd8=eventsAnd8.drop(1)
eventsAnd8.columns = new_header
eventsAnd8.rename(columns={'Name':'Event'}, inplace=True)
eventsAnd8.drop_duplicates(subset=['Event'], inplace=True)

eventsWDSF = webscraping['Events WDSF'][1:].copy()
new_header = eventsWDSF.iloc[0]
eventsWDSF = eventsWDSF.drop(1)
eventsWDSF.columns = new_header
eventsWDSF.rename(columns={'Name': 'Event'}, inplace=True)
eventsWDSF.drop_duplicates(subset=['Event'], inplace=True)

Events = webscraping['Events'][1:].copy()
new_header = Events.iloc[0]
Events = Events.drop(1)
Events.columns = new_header
Events.rename(columns={'Name':'Event'}, inplace=True)
Events.drop_duplicates(subset=['Event'], inplace=True)

combined_events = pd.concat([eventsAnd8, eventsWDSF, Events], ignore_index=True)


In [11]:
#add battle date
appendedBattles=pd.merge(appendedBattles, combined_events[['Date', 'Event']], on='Event', how='left')


In [12]:
appendedBattles.tail(2)

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,Original_DF,Battle_ID,Date
2627,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,Stiv,D-Matt,D-Matt,trivium,1524,2017-06-10 00:00:00
2628,WDSF Pan American Qualifier 2017 - Philadelphia,B-Boys Top 16 Round Robin Group A,D-Matt,Rerunn,D-Matt,trivium,1525,2017-06-10 00:00:00


Threefold Battles

In [13]:
#threefold battles
threefold = webscraping['Battles and8 Threefold']
threefold.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
threefold[['Event', 'Category']] = threefold[['Event', 'Category']].fillna(method='ffill')
threefold.head(2)

,Event,Category,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,breaker 1 vote count,...,r3j7arti,r3j7inte,r3j8over,r3j8phys,r3j8arti,r3j8inte,r3j9over,r3j9phys,r3j9arti,r3j9inte
0,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,Baruzzo,Elpablix,Baruzzo,3,3,2,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,Yoga,Faby,Faby,2,3,0,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Change new names
threefold.rename(columns = {"breaker 1": "breaker 1 (red)", "breaker 2": "breaker 2 (blue)"}, inplace=True)


In [15]:
threefold.columns

Index(['Event', 'Category', 'breaker 1 (red)', 'breaker 2 (blue)', 'winner',
       'battle rounds', 'number of judges', 'breaker 1 round wins',
       'breaker 2 round wins', 'breaker 1 vote count',
       ...
       'r3j7arti', 'r3j7inte', 'r3j8over', 'r3j8phys', 'r3j8arti', 'r3j8inte',
       'r3j9over', 'r3j9phys', 'r3j9arti', 'r3j9inte'],
      dtype='object', length=128)

In [16]:
# Using DataFrame.insert() to add a column
threefold.insert(2, "Method", ['three']*len(threefold), True)

In [17]:
threefold.head(2)

,Event,Category,Method,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,...,r3j7arti,r3j7inte,r3j8over,r3j8phys,r3j8arti,r3j8inte,r3j9over,r3j9phys,r3j9arti,r3j9inte
0,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,three,Baruzzo,Elpablix,Baruzzo,3,3,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,three,Yoga,Faby,Faby,2,3,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Create 'Date' Column with combined_events

def Date_Added(df, combined_df):
  lst = []
  for i in range(len(df)):
    lst.append(combined_df[combined_df['Event']==df['Event'].iloc[i]]['Date'].iloc[0])


  # Using DataFrame.insert() to add a column
  df.insert(0, "Date", lst, True)
  return df


In [19]:
threefold = Date_Added(threefold, appendedBattles)

In [20]:
threefold.tail()

,Date,Event,Category,Method,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,...,r3j7arti,r3j7inte,r3j8over,r3j8phys,r3j8arti,r3j8inte,r3j9over,r3j9phys,r3j9arti,r3j9inte
368,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Wigor,El Nino,Wigor,2,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Mighty Jimm,Nasty,Mighty Jimm,3,7,2,...,-4.33,-3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Daniel,Mass,Mass,3,7,1,...,3.00,3.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
371,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Quake,Flo,Quake,2,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Neosan,Lilbom,Lilbom,2,7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Trivium


---

Between each round of a battle, each judge gives a score (1-100) representing how much they believe one side outperformed the other (or scores 0 to represent a tie) in each of 6 categories (technique, variety, creativity, personality, performativity, musicality), the sum of those 6 scores determine which side the judge is voting for, and the majority of judge votes determines the winner of the round. The individual who wins the majority of the rounds in a battle then wins the battle. The judge also has a set of buttons that can be pressed (execution, form, confidence, spontaneity, repeat, bite, crash) that function as shortcuts where each button adjusts some of the scores by a certain amount. There was disagreement among judges whether these buttons should be adjusting their input scores, and as a result the usage of the buttons by different judges was inconsistent.

In [21]:
test_trivium=webscraping['Battles Trivium']
test_trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
test_trivium[['Event', 'Category']]= test_trivium[['Event', 'Category']].fillna(method='ffill') # fill values by Event's name

In [22]:
# Using DataFrame.insert() to add a column
test_trivium.insert(2, "Method", ['trivium']*len(test_trivium), True)

In [23]:
test_trivium = Date_Added(test_trivium, combined_events)

In [57]:
test_trivium.head(2)

,Date,Event,Category,Method,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,...,r5j8but2,r5j9over,r5j9tech,r5j9vari,r5j9perf,r5j9musi,r5j9crea,r5j9pers,r5j9but1,r5j9but2
0,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,trivium,Ami,671,671,3,9,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 8 Knockout,trivium,Sunny,Anti,Anti,3,9,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
combined_tv_th = pd.concat([test_trivium, threefold], ignore_index=True)

In [26]:
combined_tv_th.tail()

,Date,Event,Category,Method,breaker 1 (red),breaker 2 (blue),winner,battle rounds,number of judges,breaker 1 round wins,...,r3j6inte,r3j7phys,r3j7arti,r3j7inte,r3j8phys,r3j8arti,r3j8inte,r3j9phys,r3j9arti,r3j9inte
1894,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Wigor,El Nino,Wigor,2,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1895,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Mighty Jimm,Nasty,Mighty Jimm,3,7,2,...,0.33,-3.67,-4.33,-3.00,NaN,NaN,NaN,NaN,NaN,NaN
1896,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Daniel,Mass,Mass,3,7,1,...,0.67,2.33,3.00,3.67,NaN,NaN,NaN,NaN,NaN,NaN
1897,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Quake,Flo,Quake,2,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1898,2023-04-06,WDSF BfG International Series - Montreal - Can...,1vs1 Breaking B-Boy Top 16 KnockOut,three,Neosan,Lilbom,Lilbom,2,7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Threefold -
Components :

*  'Physical Quality',
*  'Artistic Quality',
*  'Interpretive Quality'

# Trivium -
Components :

*  'Technique',
*  'Variety',
*  'Performance',
*  'Musicality',
*  'Creativity',
*  'Personality'

Create Dataframe -'Event', 'Category', 'Round number','Name', 'Overall', 'Technique', 'Variety', 'Performance', 'Musicality', 'Creativity', 'Personality'

Dictionary


*   **Date** = date of event (datetime)
*   **Event** = name of event (string)
*   **Category** = name of round (string)
*   **Method** = Threefold or Trivium
*   **Gender** = Female or Male
*   **Round Number** = the number of each round (r1 means the first round for battle)
*   **Judge** = name of judge (string)
*   **Overall** = Sum of judge scores (float)
*   **Technique_Physical** = judge scores for techique(trivium) or physical quality(threefold) (float)
*   **Variety_Artistic** = judge scores for variety(trivium) or artistic quality(threefold) (float)
*   **Performance_Interpretive** = judge scores for performance(trivium) or interpretive quality(threefold) (float)
*   **musicalirty** = judge scores for musicality (float)
*   **Creativity** = judge scores for creativity (float)
*   **Personality** = judge scores for personality (float)





# Create a dataframe by Judges

In [27]:
columns = ['Date', 'Event', 'Category','Method', 'Gender', 'Round_Number','Judge', 'Winner_In_Round', 'Overall', 'Technique_Physical', 'Variety_Artistic', 'Performance_Interpretive', 'Musicality', 'Creativity', 'Personality']

judge_df = pd.DataFrame(columns=columns)


In [28]:
# def create_judge_df(df):


event_lst = list(combined_tv_th['Event'].unique())

total_judges_lst = [] # Save the entire judges

total_breakers_lst = [] # Save the entire judges

# If there is any change, you must run dataframe judge_df first!
# Index number is initialized
index_num = 0

for event in event_lst:
  temp_lst = [] # empty list
  temp_df = combined_tv_th[combined_tv_th['Event']==event] # Take dataframe by each event

  date = temp_df['Date'].iloc[0]

  category_lst = temp_df['Category'].unique()
  #print(category_lst)

  # string of numbers by Descending order
  category_lst = natsorted(category_lst, reverse=True)

  for category in category_lst:

    cat_temp_df = temp_df[temp_df['Category']==category] # Take dataframe by category in each event
    # print(cat_temp_df['battle rounds'].unique())
    # print('')

    if "Girls" in category :
      gender = "Female"
    else:
      gender = "Male"


    for k in range(len(cat_temp_df)):
      num_judges = cat_temp_df['number of judges'].iloc[k]
      judges_lst = list(cat_temp_df.filter(like="judge ").iloc[k, :num_judges]) # Judges list
      num_round = cat_temp_df['battle rounds'].iloc[k] # number of round

      total_judges_lst = list(set(total_judges_lst + judges_lst))

      num_breakers = 2 # number of breakers
      breakers_lst = list(cat_temp_df.filter(like="breaker ").iloc[k, :num_breakers]) # breakers list

      # convert int to str!!
      if 671 in breakers_lst:
        breakers_lst[0] = str(breakers_lst[0])
        breakers_lst[1] = str(breakers_lst[1])

      total_breakers_lst = list(set(total_breakers_lst + breakers_lst))

      for i in range(num_round):
        round = "r"+str(i+1) # Round number

        for j in range(num_judges):
          temp_lst.append(date)
          temp_lst.append(event)
          temp_lst.append(category)

          method = cat_temp_df['Method'].iloc[k]
          temp_lst.append(method)

          temp_lst.append(gender)


          temp_lst.append(round)

          temp_lst.append(judges_lst[j])

          each_r_j_num = round+'j'+str(j+1)
          over = each_r_j_num+'over'

          # if trivium data
          if method =='trivium':
            # Score list - Overall, Tech, Variety, Performance, Musicality, Creative, Personality
            tech = each_r_j_num +'tech'
            vari = each_r_j_num +'vari'
            perf = each_r_j_num +'perf'
            musi = each_r_j_num +'musi'
            crea = each_r_j_num +'crea'
            pers = each_r_j_num +'pers'

            a_l = [over, tech, vari, perf, musi, crea, pers]
            s_lst = list(cat_temp_df[a_l].iloc[k])

          else:
            s_lst = [] # Save scores(by judges)

            phys = each_r_j_num +'phys'
            arti = each_r_j_num +'arti'
            inte = each_r_j_num +'inte'

            a_l = [over, phys, arti, inte]

            s_lst = list(cat_temp_df[a_l].iloc[k]) + [np.nan]*3

          # Append winner breaker by Overall score
          # if Overall > 0, blue(breaker2)
          temp_lst.append(np.where(s_lst[0]>0, str(breakers_lst[1]), str(breakers_lst[0])))

          judge_df.loc[index_num] = temp_lst+ s_lst
          index_num = index_num + 1

          temp_lst = [] # reset the list


  # return total_breakers_lst, total_judges_lst, judge_df


In [29]:
# Sort them by ascending order
total_judges_lst.sort()
total_breakers_lst.sort()

In [58]:
#How many judges?
len(total_judges_lst)

80

In [31]:
len(total_breakers_lst)

738

In [32]:

def abs_df(df):

  # Overall > 0, blue
  e0 = pd.Series(np.where(df['Overall']>0, 'blue', 'red'))
  df = df.assign(color=e0.values)

  e = df['Overall'].abs()
  df = df.assign(Overall=e.values)

  e1 = df['Technique_Physical'].abs()
  df = df.assign(Technique_Physical=e1.values)

  e2 = df['Variety_Artistic'].abs()
  df= df.assign(Variety_Artistic=e2.values)

  e3 = df['Performance_Interpretive'].abs()
  df = df.assign(Performance_Interpretive=e3.values)

  e4 = df['Musicality'].abs()
  df = df.assign(Musicality=e4.values)

  e5 = df['Creativity'].abs()
  df = df.assign(Creativity=e5.values)

  e6 = df['Personality'].abs()
  df = df.assign(Personality=e6.values)

  return df

All scores must set the absolute values

In [33]:
# Let's take a copy to keep the orginal
judge_df_cp = judge_df.copy()

# Get absoluete values
judge_df_cp = abs_df(judge_df_cp)

In [34]:
#For information on how to plot larger datasets in Altair, see the documentation
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

Let us look deeper at each judge.

**Judge distraction / fatigue**: Moving sliders on the judging interface takes attention away from watching the battle. Do some judges move fewer sliders? Are there differences in the votes between judges who move fewer sliders versus judges who move more sliders? Presumably moving more sliders requires more mental effort in deciding more scores,so does a judge who moves more sliders show signs of fatigue over time (perhaps gradually moving fewer sliders towards the end of the event or less consistent scoring towards end of event)?

In [35]:
def count_zeros(df, method):
  df1 = df[df['Method']==method]
  total_method = len(df1)
  if method == 'trivium':
    columns = ['Technique_Physical',
       'Variety_Artistic', 'Performance_Interpretive', 'Musicality',
       'Creativity', 'Personality']
  else:
    columns = ['Technique_Physical',
       'Variety_Artistic', 'Performance_Interpretive']
  num = 0
  for i in df1.columns:
    num += df1[i][df1[i]==0].count()
  return (num, total_method)

In [36]:
# Create a count dataframe for each round
columns = ['Method', 'Round_Number', 'Percentage']
distraction_df = pd.DataFrame(columns=columns)

# Count zeros in each round


distraction_list = []

round_list = list(judge_df_cp['Round_Number'].unique())

index_num = 0

for round in round_list:

  print("Round : ", round)
  print("")

  round_df = judge_df_cp[judge_df_cp['Round_Number']==round]

  # Trivium
  num_zeros_trivium, total_trivium = count_zeros(round_df, 'trivium')

  percentage_trivium = 0
  if total_trivium != 0:
    percentage_trivium = num_zeros_trivium/(total_trivium*6)
  distraction_list.append('trivium')
  distraction_list.append(round)
  distraction_list.append(percentage_trivium)

  distraction_df.loc[index_num] = distraction_list
  index_num = index_num + 1
  distraction_list = [] # reset the list


  print('The number of zeros(Trivium) {}, the number of total components {}'.format(num_zeros_trivium, total_trivium*6))
  print('Therefore, the percentage of zeros in the round : {}'.format(percentage_trivium))


  num_zeros_three, total_threefold = count_zeros(round_df, 'three')

  percentage_threefold = 0
  if total_threefold != 0:
    percentage_threefold = num_zeros_three/(total_threefold*3)
  distraction_list.append('threefold')
  distraction_list.append(round)
  distraction_list.append(percentage_threefold)

  distraction_df.loc[index_num] = distraction_list
  index_num = index_num + 1
  distraction_list = [] # reset the list

  print('The number of zeros(Threefold) {}, the number of total components {}'.format(num_zeros_three, total_threefold*3))
  print('Therefore, the percentage of zeros in the round : {}'.format(percentage_threefold))
  print("")

Round :  r1

The number of zeros(Trivium) 20947, the number of total components 54840
Therefore, the percentage of zeros in the round : 0.38196571845368343
The number of zeros(Threefold) 419, the number of total components 4737
Therefore, the percentage of zeros in the round : 0.08845260713531772

Round :  r2

The number of zeros(Trivium) 16388, the number of total components 41838
Therefore, the percentage of zeros in the round : 0.39170132415507436
The number of zeros(Threefold) 365, the number of total components 3942
Therefore, the percentage of zeros in the round : 0.09259259259259259

Round :  r3

The number of zeros(Trivium) 2457, the number of total components 5934
Therefore, the percentage of zeros in the round : 0.41405460060667343
The number of zeros(Threefold) 100, the number of total components 909
Therefore, the percentage of zeros in the round : 0.11001100110011001

Round :  r4

The number of zeros(Trivium) 1586, the number of total components 3348
Therefore, the percent

In [37]:
distraction_df

,Method,Round_Number,Percentage
0,trivium,r1,0.381966
1,threefold,r1,0.088453
2,trivium,r2,0.391701
3,threefold,r2,0.092593
4,trivium,r3,0.414055
5,threefold,r3,0.110011
6,trivium,r4,0.473716
7,threefold,r4,0.000000
8,trivium,r5,0.550000
9,threefold,r5,0.000000


In [38]:
def distraction_altair(df):
  base = alt.Chart().mark_point().encode(
      color='Method:N',
      tooltip=['Method', 'Percentage'] # show Name and Origin in a tooltip
  ).properties(
      width=200,
      height=220
  ).properties(
      title = 'Judge Distraction'
  ).interactive()



  chart = alt.vconcat(data=df)
  for y_encoding in ['Percentage:Q']:
      row = alt.hconcat()
      for x_encoding in ['Round_Number:N']:
          row |= base.encode(x=x_encoding, y=y_encoding)
      chart &= row

  #chart

  return chart.configure_title(
      fontSize=15,
      font='Courier',
      anchor='start',
      color='gray'
  )

In [39]:
distraction_altair(distraction_df)

alt.VConcatChart(...)

In [40]:
# replace r1, r2, r3 for 1, 2, 3 to use regression
distraction_df_cp = distraction_df.copy()
distraction_df_cp = distraction_df_cp.replace({'r1': 1, 'r2': 2, 'r3': 3, 'r4': 4, 'r5': 5})

In [41]:
distraction_df_cp


,Method,Round_Number,Percentage
0,trivium,1,0.381966
1,threefold,1,0.088453
2,trivium,2,0.391701
3,threefold,2,0.092593
4,trivium,3,0.414055
5,threefold,3,0.110011
6,trivium,4,0.473716
7,threefold,4,0.000000
8,trivium,5,0.550000
9,threefold,5,0.000000


Trivium dataset


In [42]:
only_tri = distraction_df_cp[distraction_df_cp['Method']=='trivium']
chart = alt.Chart(only_tri).mark_point().encode(
    x='Round_Number',
    y='Percentage'
)

chart + chart.transform_regression('Round_Number', 'Percentage').mark_line()

alt.LayerChart(...)

In [43]:
x = list(only_tri['Round_Number'])
y = list(only_tri['Percentage'])

slope, intercept, r, p, std_err = stats.linregress(x, y)

print('The slope {}, intercept {}'.format(slope, intercept))

print('The r vaule {}, p-value {}'.format(r, p))


The slope 0.041808289007256486, intercept 0.316862591848318
The r vaule 0.94472694110823, p-value 0.015469285878185577


Threefold dataset

In [44]:
only_three = distraction_df_cp[distraction_df_cp['Method']=='threefold']
only_three = only_three.head(3)
chart = alt.Chart(only_three).mark_point().encode(
    x='Round_Number',
    y='Percentage'
)

chart + chart.transform_regression('Round_Number', 'Percentage').mark_line()

alt.LayerChart(...)

In [45]:
x = list(only_three['Round_Number'])
y = list(only_three['Percentage'])

slope, intercept, r, p, std_err = stats.linregress(x, y)

print('The slope {}, intercept {}'.format(slope, intercept))

print('The r vaule {}, p-value {}'.format(r, p))


The slope 0.010779196982396148, intercept 0.0754603396445478
The r vaule 0.9421995919404375, p-value 0.2175078654889404


Judge Fatique (Variance)

In [46]:
judge_df_cp.head(2)

,Date,Event,Category,Method,Gender,Round_Number,Judge,Winner_In_Round,Overall,Technique_Physical,Variety_Artistic,Performance_Interpretive,Musicality,Creativity,Personality,color
0,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 32 PreQualifier,trivium,Female,r1,Kujo,Furia,6.60,5.2,0.0,2.0,0.93,1.8,0.67,blue
1,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 32 PreQualifier,trivium,Female,r1,Valentine,Furia,7.33,0.0,0.0,4.0,0.00,2.0,1.33,blue


In [47]:
def stat_groupby_Round(df1):

  #calculate mean and standard deviation of points, grouped by judges
  df1 = df1.groupby(['Round_Number'], as_index=False).agg(mean_overall=('Overall', 'mean'),
                                                              var_overall=('Overall', 'var'),
                                                              min_overall=('Overall', 'min'),
                                                              max_overall=('Overall', 'max'),
                                                              median_overall=('Overall', 'median'))

  print(tabulate(df1, headers='keys', tablefmt='psql'))

  return df1


In [48]:
judge_df_cp.columns

Index(['Date', 'Event', 'Category', 'Method', 'Gender', 'Round_Number',
       'Judge', 'Winner_In_Round', 'Overall', 'Technique_Physical',
       'Variety_Artistic', 'Performance_Interpretive', 'Musicality',
       'Creativity', 'Personality', 'color'],
      dtype='object')

In [49]:
round_group_df = stat_groupby_Round(judge_df_cp)

+----+----------------+----------------+---------------+---------------+---------------+------------------+
|    | Round_Number   |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+----------------+----------------+---------------+---------------+---------------+------------------|
|  0 | r1             |        6.92489 |      58.6663  |          0    |        100    |            4.8   |
|  1 | r2             |        6.94542 |      61.2022  |          0    |        100    |            4.87  |
|  2 | r3             |        6.73495 |      72.2906  |          0.07 |        100    |            4.47  |
|  3 | r4             |        5.05692 |      15.8978  |          0.07 |         29.93 |            4     |
|  4 | r5             |        3.046   |       9.30751 |          0.53 |         12.4  |            2.265 |
+----+----------------+----------------+---------------+---------------+---------------+------------------+


Null hypothesis : There is the same overall score between rounds

In [50]:
# import statsmodels sm,ols and pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# create a model with statsmodels ols and fit data into the model
model = ols(formula='Overall ~ Round_Number', data=judge_df_cp).fit()

# call statsmodels stats.anova_lm and pass model as an argument to perform ANOVA
anova_result = sm.stats.anova_lm(model, type=2)

print(anova_result)

                   df        sum_sq     mean_sq         F        PR(>F)
Round_Number      4.0  2.229641e+03  557.410348  9.395162  1.388403e-07
Residual      20871.0  1.238266e+06   59.329512       NaN           NaN


In [51]:
p_value = anova_result['PR(>F)'].iloc[0]
alpha = 0.05

if p_value < alpha:
  print("Reject the null hypothesis; there is a significant difference between each round.")
else:
  print("Fail to reject the null hypothesis; there is no significant difference between each round.")


Reject the null hypothesis; there is a significant difference between each round.


In [52]:
# call statsmodels' pairwise_tukeyhsd to perform multiple comparisons test
# assign judge_df_cp Overall and Round_Number columns and set alpha to 0.05
multiple_comp_result = pairwise_tukeyhsd(endog=judge_df_cp['Overall'],
                                         groups=judge_df_cp['Round_Number'],
                                        alpha=0.05)
print(multiple_comp_result.summary())

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    r1     r2   0.0205 0.9998 -0.2868  0.3279  False
    r1     r3  -0.1899  0.919 -0.8088  0.4289  False
    r1     r4   -1.868    0.0 -2.7804 -0.9556   True
    r1     r5  -3.8789 0.1615 -8.5819  0.8241  False
    r2     r3  -0.2105 0.8918  -0.839   0.418  False
    r2     r4  -1.8885    0.0 -2.8075 -0.9695   True
    r2     r5  -3.8994 0.1577 -8.6037  0.8048  False
    r3     r4   -1.678 0.0002 -2.7425 -0.6136   True
    r3     r5  -3.6889 0.2092 -8.4238  1.0459  False
    r4     r5  -2.0109 0.7813  -6.793  2.7711  False
----------------------------------------------------


**Variance of scores**: Do some categories show more variance among judges' submitted scores versus other categories? Would that suggest that judges have less agreement on the definitions of some categories?

In [59]:
judge_df.shape

(20876, 15)

In [60]:
len(judge_df['Category'].unique())

95

In [61]:
judge_df.head()

,Date,Event,Category,Method,Gender,Round_Number,Judge,Winner_In_Round,Overall,Technique_Physical,Variety_Artistic,Performance_Interpretive,Musicality,Creativity,Personality
0,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 32 PreQualifier,trivium,Female,r1,Kujo,Furia,6.60,5.2,0.0,-2.0,0.93,1.8,0.67
1,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 32 PreQualifier,trivium,Female,r1,Valentine,Furia,7.33,0.0,0.0,4.0,0.00,2.0,1.33
2,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 32 PreQualifier,trivium,Female,r1,Spin,Furia,5.27,0.0,0.0,1.8,1.60,0.0,1.87
3,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 32 PreQualifier,trivium,Female,r1,Katsu One,Luma,-3.13,-2.0,-0.4,-2.0,1.07,-1.4,1.60
4,2023-02-24,WDSF BfG World Series - Kitakyushu 2023 - Japan,B-Girls Top 32 PreQualifier,trivium,Female,r1,Kim,Furia,3.80,0.0,0.0,2.2,1.60,0.0,0.00


In [62]:
def stat_groupby_category(df1):

  #calculate mean and standard deviation of points, grouped by judges
  df1 = df1.groupby(['Category'], as_index=False).agg(mean_overall=('Overall', 'mean'),
                                                              var_overall=('Overall', 'var'),
                                                              min_overall=('Overall', 'min'),
                                                              max_overall=('Overall', 'max'),
                                                              median_overall=('Overall', 'median'))

  print(tabulate(df1, headers='keys', tablefmt='psql'))
  return df1

In [68]:
def stat_groupby_judge(df1):

  #calculate mean and standard deviation of points, grouped by judges
  df1 = df1.groupby(['Judge'], as_index=False).agg(mean_overall=('Overall', 'mean'),
                                                              var_overall=('Overall', 'var'),
                                                              min_overall=('Overall', 'min'),
                                                              max_overall=('Overall', 'max'),
                                                              median_overall=('Overall', 'median'))

  print(tabulate(df1, headers='keys', tablefmt='psql'))

  return df1



In [63]:
cat_output = stat_groupby_category(judge_df_cp)

+----+------------------------------------------------+----------------+---------------+---------------+---------------+------------------+
|    | Category                                       |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+------------------------------------------------+----------------+---------------+---------------+---------------+------------------|
|  0 | 1vs1 B-Boy (SBL) Top 16 KnockOut               |       19.1563  |     407.361   |          0.33 |         87    |           11     |
|  1 | 1vs1 B-Boy Top 32 KnockOut                     |        7.30935 |      34.1467  |          0.33 |         32.67 |            6.33  |
|  2 | 1vs1 B-Boys Top 16 RoundRobin Group A          |       15.6833  |     305.067   |          0.33 |         75    |            7.5   |
|  3 | 1vs1 B-Boys Top 16 RoundRobin Group B          |       14.6437  |     240.186   |          0.67 |         59    |            7.665 |
|  4 | 1vs1 B-Boys T

Find outliers (more variance)


---

The interquartile range (IQR) is a measure of the spread of the middle 50% of the data. The IQR can be calculated as the difference between the 75th percentile and the 25th percentile of the dataset. Any data point outside the range of 1.5 times the IQR below the 25th percentile or above the 75th percentile can be considered an outlier.

To identify outliers using the IQR, we can use the quantile() function in pandas to calculate the 25th and 75th percentiles of the dataset. We can then calculate the IQR and use it to identify outliers.

In [64]:
# calculate IQR for column Height
Q1 = cat_output['var_overall'].quantile(0.25)
Q3 = cat_output['var_overall'].quantile(0.75)
IQR = Q3 - Q1

# identify outliers
threshold = 1.5
outliers = cat_output[(cat_output['var_overall'] < Q1 - threshold * IQR) | (cat_output['var_overall'] > Q3 + threshold * IQR)]

In [75]:
outliers

,Category,mean_overall,var_overall,min_overall,max_overall,median_overall
0,1vs1 B-Boy (SBL) Top 16 KnockOut,19.156296,407.360551,0.33,87.00,11.000
4,1vs1 B-Boys Top 16 RoundRobin Group C,17.650000,323.234827,0.33,73.00,9.500
7,1vs1 B-Boys Top 8 Knockout,23.073667,817.156841,0.67,100.00,11.000
9,1vs1 B-Girl (SBL/U17) Top 4 KnockOut,15.266000,379.624052,2.33,78.00,6.330
11,1vs1 B-Girls Top 16 RoundRobin Group A,17.939000,411.034755,0.33,90.33,9.330
12,1vs1 B-Girls Top 16 RoundRobin Group B,20.906167,394.341187,0.67,72.67,10.835
14,1vs1 B-Girls Top 16 RoundRobin Group D,20.438833,497.992980,2.00,81.67,8.835
17,1vs1 BBoy 12-15 U Top 16 KnockOut,19.537222,338.908495,0.33,81.67,13.670
18,1vs1 BBoy 16-18 C\tTop 4 KnockOut,19.685556,401.176885,0.33,67.33,10.000
20,1vs1 BBoy 19/OL U\tTop 8 KnockOut,21.308519,512.927096,1.00,78.33,10.330


In [71]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'][0]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'][0]
)

chart2

+----+---------+----------------+---------------+---------------+---------------+------------------+
|    | Judge   |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+---------+----------------+---------------+---------------+---------------+------------------|
|  0 | AT      |        7.76556 |       44.2065 |          0.33 |         26    |             5.67 |
|  1 | RoxRite |       10.6781  |       37.1966 |          1.33 |         23.33 |            10.33 |
|  2 | Thesis  |       39.0252  |      552.675  |          3    |         87    |            37.33 |
+----+---------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)

In [77]:
outliers['Category'].iloc[1]

'1vs1 B-Boys Top 16 RoundRobin Group C'

In [78]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'].iloc[1]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'].iloc[1]
)

chart2

+----+---------+----------------+---------------+---------------+---------------+------------------+
|    | Judge   |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+---------+----------------+---------------+---------------+---------------+------------------|
|  0 | Bojin   |        6.75    |      16.6949  |          0.67 |         11.67 |            8.5   |
|  1 | Danilo  |       38.7775  |     443.972   |          7.33 |         73    |           32.835 |
|  2 | Jess    |        4.38833 |       2.25852 |          1    |          6.33 |            4.835 |
|  3 | Max     |        9.0825  |      80.8473  |          0.33 |         33.33 |            7     |
|  4 | Spin    |       29.2517  |     154.61    |          3.67 |         44.67 |           30.5   |
+----+---------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)

In [79]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'].iloc[2]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'].iloc[2]
)

chart2

+----+---------+----------------+---------------+---------------+---------------+------------------+
|    | Judge   |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+---------+----------------+---------------+---------------+---------------+------------------|
|  0 | Bojin   |        8.38889 |       19.9933 |          3    |            18 |            7.335 |
|  1 | Danilo  |       34.26    |      994.552  |          2.33 |           100 |           23     |
|  2 | Jess    |        5.05444 |       16.703  |          0.67 |            16 |            3.83  |
|  3 | Max     |       21.8506  |     1305.19   |          1.33 |           100 |            6.665 |
|  4 | Spin    |       45.8144  |      687.86   |         21.33 |           100 |           40.5   |
+----+---------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)

In [80]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'].iloc[3]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'].iloc[3]
)

chart2

+----+-----------+----------------+---------------+---------------+---------------+------------------+
|    | Judge     |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+-----------+----------------+---------------+---------------+---------------+------------------|
|  0 | Feel      |         31.832 |     677.552   |          6    |         78    |           21.165 |
|  1 | My-Key77  |          2.932 |       0.36524 |          2.33 |          4.33 |            3     |
|  2 | OG Samson |         11.034 |      51.4602  |          4.33 |         27.67 |            7.835 |
+----+-----------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)

In [81]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'].iloc[4]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'].iloc[4]
)

chart2

+----+---------+----------------+---------------+---------------+---------------+------------------+
|    | Judge   |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+---------+----------------+---------------+---------------+---------------+------------------|
|  0 | Bojin   |        7.05583 |      18.3986  |          1    |         12.33 |            8.17  |
|  1 | Danilo  |       35.8617  |     650.191   |         12    |         90.33 |           28.335 |
|  2 | Jess    |        4.44333 |       7.52277 |          0.33 |          9.33 |            4.5   |
|  3 | Max     |        5.66667 |       5.3489  |          1.67 |          9.33 |            6.165 |
|  4 | Spin    |       36.6675  |     297.908   |          1.67 |         54.33 |           45.835 |
+----+---------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)

In [82]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'].iloc[5]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'].iloc[5]
)

chart2

+----+---------+----------------+---------------+---------------+---------------+------------------+
|    | Judge   |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+---------+----------------+---------------+---------------+---------------+------------------|
|  0 | Bojin   |        7.36167 |      15.2527  |          0.67 |         13.33 |            7.17  |
|  1 | Danilo  |       46.2242  |     226.914   |         25    |         72.67 |           41.335 |
|  2 | Jess    |        4.945   |       6.76843 |          0.67 |          9.67 |            4.67  |
|  3 | Max     |        8.2775  |      17.4269  |          2.67 |         16.67 |            8.83  |
|  4 | Spin    |       37.7225  |     188.938   |         15.67 |         53    |           45.835 |
+----+---------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)

In [83]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'].iloc[6]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'].iloc[6]
)

chart2

+----+---------+----------------+---------------+---------------+---------------+------------------+
|    | Judge   |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+---------+----------------+---------------+---------------+---------------+------------------|
|  0 | Bojin   |        6.30583 |       5.24181 |          2.33 |          9    |            6.665 |
|  1 | Danilo  |       46.8058  |     785.415   |          9.67 |         81.67 |           46     |
|  2 | Jess    |        5.44417 |       4.87812 |          2    |          9    |            5.335 |
|  3 | Max     |        7.72167 |      36.4844  |          2.33 |         23    |            6     |
|  4 | Spin    |       35.9167  |     179.663   |         15.33 |         52.33 |           41.335 |
+----+---------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)

In [84]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'].iloc[7]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'].iloc[7]
)

chart2

+----+----------+----------------+---------------+---------------+---------------+------------------+
|    | Judge    |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+----------+----------------+---------------+---------------+---------------+------------------|
|  0 | Dom-K    |       17.5189  |       87.3165 |          3.67 |         38.33 |           14.835 |
|  1 | Kadoer   |        7.72222 |       38.2622 |          0.33 |         22    |            6.165 |
|  2 | Philgood |       33.3706  |      565.912  |          1    |         81.67 |           30.335 |
+----+----------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)

In [85]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'].iloc[8]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'].iloc[8]
)

chart2

+----+----------+----------------+---------------+---------------+---------------+------------------+
|    | Judge    |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+----------+----------------+---------------+---------------+---------------+------------------|
|  0 | Dom-K    |       13.055   |      105.934  |          3.67 |         26    |            9.165 |
|  1 | Kadoer   |        3.27833 |       15.4564 |          0.33 |         10.67 |            1.5   |
|  2 | Philgood |       42.7233  |      229.931  |         22.67 |         67.33 |           40.67  |
+----+----------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)

In [86]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'].iloc[9]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'].iloc[9]
)

chart2

+----+----------+----------------+---------------+---------------+---------------+------------------+
|    | Judge    |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+----------+----------------+---------------+---------------+---------------+------------------|
|  0 | Dom-K    |       13.3144  |       75.5638 |          4.67 |         31.33 |           10.165 |
|  1 | Kadoer   |        7.31556 |       20.0608 |          1    |         16.33 |            7.17  |
|  2 | Philgood |       43.2956  |      716.649  |          4    |         78.33 |           47     |
+----+----------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)

In [87]:
outlier_df0 = judge_df_cp[judge_df_cp['Category']==outliers['Category'].iloc[10]]

outlier_df00 = stat_groupby_judge(outlier_df0)

chart2 = alt.Chart(outlier_df00).mark_circle().encode(
    x='Judge:N',
    y='mean_overall:Q',
).properties(
    title='Total Mean Overall Score per Judge - '+outliers['Category'].iloc[10]
)

chart2

+----+----------+----------------+---------------+---------------+---------------+------------------+
|    | Judge    |   mean_overall |   var_overall |   min_overall |   max_overall |   median_overall |
|----+----------+----------------+---------------+---------------+---------------+------------------|
|  0 | Dom-K    |         24.2   |      109.466  |          5.33 |         37    |           25.83  |
|  1 | Kadoer   |         13.066 |       38.1453 |          4.33 |         21.33 |           13.665 |
|  2 | Philgood |         39.999 |      846.791  |          1.67 |         78    |           47.5   |
+----+----------+----------------+---------------+---------------+---------------+------------------+


alt.Chart(...)